In [2]:
import pandas as pd
import gzip
import json
import torch.utils.data as data

In [43]:
from fastai.text.all import text_classifier_learner, TextDataLoaders, AWD_LSTM, Perplexity, accuracy, language_model_learner,untar_data,URLs

In [3]:
def load_data(file_name, head = None):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
            
            # break if reaches the 100th line
            if (head is not None) and (count > head):
                break
    return data

In [9]:
# df_review = pd.DataFrame(load_data(r"F:\projects\PlayGround\wip\data\Pet_Supplies.json.gz"))
# df_meta = pd.DataFrame(load_data(r"data\meta_Pet_Supplies.json.gz"))
# totw_dog_food_asin = (
#     df_meta
#     .explode("category")
#     .loc[
#     lambda x: 
#         (x.brand == "Taste of the Wild") &
#         (x.category.isin(["Food"]))
#     ]
# ).asin.unique()
# df_totw = df_review.loc[lambda x: (x.asin.isin(totw_dog_food_asin)) & (~x.reviewText.isna())]
# df_totw.to_pickle(r"F:\projects\PlayGround\wip\data\df_totw.pickle")

In [26]:
#__________________________________________________________#

In [26]:
df_totw = pd.read_pickle(r"F:\projects\PlayGround\wip\data\df_totw.pickle").iloc[:1000]

In [27]:
df_totw.head(1)

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
1404950,5.0,22,False,"01 12, 2009",A3GBY9HEHC767B,B000W5QSYA,Lori A. Witkop,"After trying MANY other foods I finally found this one! I've found the food for us! It's LESS expensive than most the other ""premium"" dog foods, and has better or equivalent ingredients! My Black Lab Jade just would stick her nose up at other foods, but she goes crazy over this stuff, even has turned down canned food for it! Both she and our four month old Golden Retriever's coats are softer than ever, and after two months time they still love the taste. Never had a problem with their stools or throwing up. AND, what I like best, is it's ALL stages of life, I can have both puppy and ...","DOG TESTED, MOTHER APPROVED!!!!!!!",1231718400,NaN,NaN


In [28]:
df_rev = df_totw.loc[:,('overall', 'reviewText')]

In [29]:
df_data = TextDataLoaders.from_df(df_rev, text_col='reviewText', label_col='overall', valid_pct=0.2)

Due to IPython and Windows limitation, python multiprocessing isn't available now.
So `n_workers` has to be changed to 0 to avoid getting stuck


In [30]:
df_data.show_batch(max_n=2)

,text,category
0,xxbos xxmaj this dog food is xxup xxunk . i have been using xxmaj blue xxmaj buffalo for a couple of years now for my 2 xxmaj labs . xxmaj my dogs are n't too xxunk on the taste of the xxmaj blue xxmaj buffalo but i love the fact that it 's 100 % all natural . xxmaj so i wanted to give this food a try since it seems to be all natural as well . xxmaj upon xxunk my dogs saw the bag and could n't want to get into it fast enough . i opened the bag and mixed the food 50 / 50 with their other food to xxunk them into it . xxmaj both dogs loved the taste as they picked through the food eating only this new food . \n▁ xxmaj about 2 hours later both dogs started vomiting all over my xxunk,1.0
1,"xxbos xxmaj we were using xxmaj taste of the xxmaj wild for about 3 years and it was great grain - free food and rated very high . xxmaj however , something changed in the xxmaj summer / xxmaj xxunk 2017 . [ let me first say our xxmaj cocker xxmaj spaniel was on xxunk for xxunk and liver xxunk to xxunk side effects of xxunk xxunk - so i understand this is not xxup all food xxunk . xxmaj liver xxunk were out of xxunk in xxmaj may 2017 , but after liver xxunk , his liver tested the best yet at the end of xxmaj xxunk 2017 . ] \n xxmaj we opened a new bag of xxup totw at the end of xxmaj xxunk , and within a week he started having xxunk - xxunk diarrhea with blood and loss of xxunk of xxunk xxunk . xxmaj",1.0


In [31]:
learn = text_classifier_learner(df_data, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [32]:
learn.fine_tune(1, 1e-2)

In [39]:
learn.predict("I love this product")

('5.0', tensor(4), tensor([0.0583, 0.1012, 0.1897, 0.2387, 0.4122]))

In [44]:
path = untar_data(URLs.IMDB)


In [47]:
learn = language_model_learner(df_data, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

In [51]:
learn.show_results()

f:\projects\PlayGround\wip\venv\lib\site-packages\torch\amp\autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
f:\projects\PlayGround\wip\venv\lib\site-packages\torch\cuda\amp\grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


ValueError: Expected input batch_size (20864) to match target batch_size (64).